In [2]:
from openbabel import openbabel

In [3]:
from rdkit import Chem
from rdkit.Chem import rdBase
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import Crippen

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss

In [5]:
import sys
from rdkit import Chem
from rdkit.Chem import Descriptors, Crippen, Lipinski
from rdkit.Chem import PandasTools
import pandas as pd
from sklearn.linear_model import LinearRegression
from collections import namedtuple

In [6]:
FormoseAmmMatches = './MatchesFiles/FormoseAmmMatches.tsv'
FormoseMatches = './MatchesFiles/FormoseMatches.tsv'
GlucoseAmmMatches = './MatchesFiles/GlucoseAmmMatches.tsv'
GlucoseMatches = './MatchesFiles/GlucoseMatches.tsv'
PyruvicAcidMatches = './MatchesFiles/PyruvicAcidMatches.tsv'

## Energy Minimisation

In [7]:
from rdkit.Chem import PandasTools

Minimise energy of matches

In [8]:
def minimise(matched_data):
    matches = pd.read_csv(matched_data, sep='\t')
    PandasTools.AddMoleculeColumnToFrame(matches,'Smiles','Molecule')
    PandasTools.WriteSDF(matches, 'pp_out.sdf', molColName='Molecule', properties=list(matches.columns))
    !obabel pp_out.sdf -O Minimised.sdf --minimize --ff MMFF94

In [9]:
#%%time
#minimise(PyruvicAcidMatches)

Calculate cLogP, cLogS, H-Acceptors, H-Donors, Polar Surface Area, Druglikeness, No. Heteroatoms, VDW volume using DataWarrior 

In [10]:
FormoseAmmDescriptors = './DescriptorsFiles/FormoseAmmDescriptors.txt'
FormoseDescriptors = './DescriptorsFiles/FormoseDescriptors.txt'
GlucoseAmmDescriptors = './DescriptorsFiles/GlucoseAmmDescriptors.txt'
GlucoseDescriptors = './DescriptorsFiles/GlucoseDescriptors.txt'
PyruvicAcidDescriptors = './DescriptorsFiles/PyruvicAcidDescriptors.txt'

In [11]:
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors

Calculate Molecular Weight, Molecular Formula, No. Carbons, No. Hydrogens, No. Oxygens, Heteroatoms/Carbons, H/C, O/C using rdkit 

In [12]:
def split(word):
    return[char for char in word]

In [13]:
def molweight(array, smiles_position):
    Smi = array[smiles_position]
    mol = Chem.MolFromSmiles(Smi)
    weight = Descriptors.ExactMolWt(mol)
    return(weight)

In [14]:
def molformula(array, smiles_position):
    Smi = array[smiles_position]
    mol = Chem.MolFromSmiles(Smi)
    molformula = rdMolDescriptors.CalcMolFormula(mol)
    return(molformula)

In [15]:
def mol_traits(df, smiles_position):
    df['Molecular Weight'] = df.apply(molweight, axis=1, raw=True, result_type='expand', args=[smiles_position])
    df['Molecular Formula'] = df.apply(molformula, axis=1, raw=True, result_type='expand', args=[smiles_position])
    return(df)

In [16]:
def atom_finder(array, atom, molformula_position):
    elements = ['C', 'H', 'N', 'O']
    molformula = array[molformula_position]
    letters = split(molformula)
    dummy = ''
    for i in range(len(letters)):
        if letters[i] == atom:
            if i == len(letters)-1 or letters[i+1] in elements:
                dummy = 1
                break
            else:
                dummy += letters[i+1]
                if i+2 <= len(letters)-1:
                    if letters[i+2] not in elements:
                        dummy += letters[i+2]
    #value = float(dummy)
    return(dummy)

In [17]:
def num_atoms(df, molformula_position):
    df['No. Carbons'] = df.apply(atom_finder, axis=1, raw=True, result_type='expand', args=['C', molformula_position])
    df['No. Hydrogens'] = df.apply(atom_finder, axis=1, raw=True, result_type='expand', args=['H', molformula_position])
    df['No. Oxygens'] = df.apply(atom_finder, axis=1, raw=True, result_type='expand', args=['O', molformula_position])
    return(df)

In [18]:
def calc_descriptors(df, smiles_position, molformula_position):
    df = mol_traits(df, smiles_position)
    df = num_atoms(df, molformula_position)
    return(df)

In [19]:
def make_het_carbon(array):
    NumCarbons = float(array[14])
    try:
        NumHeteroatoms = float(array[10])
        ratio = NumHeteroatoms/NumCarbons
    except:
        ratio = 'NaN'
    return(ratio)

In [20]:
def make_hyd_carbon(array):
    NumCarbons = float(array[14])
    NumHydrogens = float(array[15])
    ratio = NumHydrogens/NumCarbons
    return(ratio)

In [21]:
def make_oxy_carbon(array):
    NumCarbons = float(array[14])
    try:
        NumOxygens = float(array[16])
        ratio = NumOxygens/NumCarbons
    except:
        ratio = 'NaN'
    return(ratio)   

In [22]:
def apply_ratios(df):
    df['Heteroatoms/Carbons'] = df.apply(make_het_carbon, axis=1, raw=True, result_type='expand')
    df['H/C'] = df.apply(make_hyd_carbon, axis=1, raw=True, result_type='expand')
    df['O/C'] = df.apply(make_oxy_carbon, axis=1, raw=True, result_type='expand')
    return(df)

Calculate ESOL

In [23]:
class ESOLCalculator:
    def __init__(self):
        self.aromatic_query = Chem.MolFromSmarts("a")
        self.Descriptor = namedtuple("Descriptor", "mw logp rotors ap")

    def calc_ap(self, mol):
        """
        Calculate aromatic proportion #aromatic atoms/#atoms total
        :param mol: input molecule
        :return: aromatic proportion
        """
        matches = mol.GetSubstructMatches(self.aromatic_query)
        return len(matches) / mol.GetNumAtoms()

    def calc_esol_descriptors(self, mol):
        """
        Calcuate mw,logp,rotors and aromatic proportion (ap)
        :param mol: input molecule
        :return: named tuple with descriptor values
        """
        mw = Descriptors.MolWt(mol)
        logp = Crippen.MolLogP(mol)
        rotors = Lipinski.NumRotatableBonds(mol)
        ap = self.calc_ap(mol)
        return self.Descriptor(mw=mw, logp=logp, rotors=rotors, ap=ap)

    def calc_esol_orig(self, mol):
        """
        Original parameters from the Delaney paper, just here for comparison
        :param mol: input molecule
        :return: predicted solubility
        """
        # just here as a reference don't use this!
        intercept = 0.16
        coef = {"logp": -0.63, "mw": -0.0062, "rotors": 0.066, "ap": -0.74}
        desc = self.calc_esol_descriptors(mol)
        esol = intercept + coef["logp"] * desc.logp + coef["mw"] * desc.mw + coef["rotors"] * desc.rotors \
               + coef["ap"] * desc.ap
        return esol

    def calc_esol(self, mol):
        """
        Calculate ESOL based on descriptors in the Delaney paper, coefficients refit for the RDKit using the
        routine refit_esol below
        :param mol: input molecule
        :return: predicted solubility
        """
        intercept = 0.26121066137801696
        coef = {'mw': -0.0066138847738667125, 'logp': -0.7416739523408995, 'rotors': 0.003451545565957996, 'ap': -0.42624840441316975}
        desc = self.calc_esol_descriptors(mol)
        esol = intercept + coef["logp"] * desc.logp + coef["mw"] * desc.mw + coef["rotors"] * desc.rotors \
               + coef["ap"] * desc.ap
        return esol

In [24]:
def calculate_esol(array, smiles_position):
    esol_calculator = ESOLCalculator()
    Smi = array[smiles_position]
    mol = Chem.MolFromSmiles(Smi)
    esol = esol_calculator.calc_esol(mol)
    return(esol)

In [25]:
def add_esol(df, smiles_position):
    df['ESOL'] = df.apply(calculate_esol, axis=1, raw=True, result_type='expand', args=[smiles_position])
    return(df)

In [26]:
def calculate_descriptors(path, smiles_position, molformula_position, name):
    df = pd.read_csv(path, sep='\t')
    df = calc_descriptors(df, smiles_position, molformula_position)
    df = apply_ratios(df)
    df = add_esol(df, smiles_position)
    df.to_csv(f'{name}Descriptors.tsv', header=None, index=None, sep='\t', mode='a')
    return(df)

In [33]:
%%time
df = calculate_descriptors(PyruvicAcidDescriptors, 1, 13, 'PyruvicAcid')

CPU times: user 3.67 s, sys: 38.4 ms, total: 3.71 s
Wall time: 3.73 s


In [34]:
df

,Generation,Smiles,Inchi,Energy,cLogP,cLogS,H-Acceptors,H-Donors,Polar Surface Area,Druglikeness,...,VDW Volume,Molecular Weight,Molecular Formula,No. Carbons,No. Hydrogens,No. Oxygens,Heteroatoms/Carbons,H/C,O/C,ESOL
0,G1,C(C(CO)O)(O)=O,RBNPOMFGQQGHHO,26.22050,-1.7932,0.177,4,3,77.76,-0.24550,...,94.172,106.026609,C3H6O4,3,6,4,1.333333,2.000000,1.333333,0.735263
1,G2,C(C(CO)(C(C)O)O)(O)=O,WKRKESUWTIYINN,58.25220,-1.9800,0.197,5,4,97.99,0.77366,...,141.530,150.052823,C5H10O5,5,10,5,1.000000,2.000000,1.000000,0.632029
2,G3,C(C(C(CC(O)=O)O)C(C)O)=O,YTCZJXDURLPWNQ,-14.80950,-1.4458,-0.609,5,3,94.83,-2.13140,...,168.120,176.068473,C7H12O5,7,12,5,0.714286,1.714286,0.714286,-0.158288
3,G3,C(C(C=O)C(CC(C)O)O)(O)=O,IHSFWOVWMRACIR,-27.14680,-1.4458,-0.609,5,3,94.83,-5.09350,...,172.240,176.068473,C7H12O5,7,12,5,0.714286,1.714286,0.714286,-0.158288
4,G3,C(C(C(C)O)C(C(CO)=O)O)=O,CPUSMWJJBISCJI,38.09790,-2.1492,-0.279,5,3,94.83,-2.08150,...,172.050,176.068473,C7H12O5,7,12,5,0.714286,1.714286,0.714286,0.519082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4966,G6,C(C(CCO)O)=CC(O)=O,VXPJFXNRJWTXQI,-32.60870,-0.6822,-0.405,4,3,77.76,-0.77902,...,144.770,146.057909,C6H10O4,6,10,4,0.666667,1.666667,0.666667,-0.224666
4967,G6,C(C(CCO)O)=C(C(O)=O)C=O,NWSYMZZMMOJWHB,-23.52460,-1.2148,-0.237,5,3,94.83,-3.13860,...,161.410,174.052823,C7H10O5,7,10,5,0.714286,1.428571,0.714286,-0.086882
4968,G6,C=CC(C(C=CC(O)=O)O)O,URGIIXSTFYWKTB,9.29334,-0.5084,-0.722,4,3,77.76,-3.58150,...,158.110,158.057909,C7H10O4,7,10,4,0.571429,1.428571,0.571429,-0.426110
4969,G6,C(C(C(CO)O)O)=CC(O)=O,JCOCWZYTGFRYRY,14.07940,-1.7040,-0.006,5,4,97.99,0.62537,...,153.190,162.052823,C6H10O5,6,10,5,0.833333,1.666667,0.833333,0.432850
